In [1]:
import cv2 as cv
import numpy as np

In [2]:
# Opencv DNN
net = cv.dnn.readNet("dnn_model/yolov4-tiny.weights", 'dnn_model/yolov4-tiny.cfg')
model = cv.dnn_DetectionModel(net)
model.setInputParams(size=(320,320), scale=1/255)

# Load class list
classes = []
with open('dnn_model/classes.txt', 'r') as file_object:
    for class_name in file_object.readlines():
        class_name = class_name.strip()
        classes.append(class_name)
        

In [5]:
# Initialize the camera
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720)
# Full HD 1920 x 1080

button_person = False

def click_button(event, x, y, flags, params):
    
    global button_person
    
    if event == cv.EVENT_LBUTTONDOWN:
        polygon = np.array([[(20, 20), (210, 20), (210, 70), (20, 70)]])
        
        is_inside = cv.pointPolygonTest(polygon, (x, y), False)
        if is_inside > 0:
            if button_person is False:
                button_person = True
            else:
                button_person = False


# Create window
cv.namedWindow('Frame')
cv.setMouseCallback('Frame', click_button)


while True:
    #Get frame
    ret, frame = cap.read()

    # Object Detection
    (class_ids, scores, bboxes)  = model.detect(frame)
    for class_id, score, bbox in zip(class_ids, scores, bboxes):
        (x, y, w, h) = bbox
        
        if  classes[class_id] == 'person' and button_person is True:
            cv.putText(frame, classes[class_id], (x, y-10), cv.FONT_HERSHEY_PLAIN, 2, (253, 144, 100), 3)
            cv.rectangle(frame, (x,y), (x + w, y + h), (233, 174, 90), thickness = 2)
    
    # Create button
    polygon = np.array([[(20, 20), (210, 20), (210, 70), (20, 70)]]) 
    cv.fillPoly(frame, polygon, (0, 0, 200))
    cv.putText(frame, 'Person', (30, 60), cv.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)
    
    
    cv.imshow('Frame', frame)
    if cv.waitKey(1) & 0xFF==ord('d'):
        break
        
cap.release()
cv.destroyAllWindows()